# seed generator = Faker

MySQL Workbench

If you want to avoid writing sql, you can also do it in MySQL Workbench by right clicking on the table, choose "Alter Table ..." in the menu.

When the table structure view opens, go to tab "Options" (on the lower bottom of the view), and set "Auto Increment" field to the value of the next autoincrement number.

Don't forget to hit "Apply" when you are done with all changes.

In [1]:
import pickle as pk
import pandas as pd

In [2]:
import MySQLdb as db
from faker import Faker
fake = Faker()

In [3]:
data = pd.read_pickle('dataset/training_set.pkl')

In [4]:
db = db.connect(
    '127.0.0.1',
    'root',
    '5555',
    'news_rec',
    charset='utf8'
)

In [5]:
# article to DB
curs = db.cursor()
for idx, val in data.iterrows():
    area = val['area']
    title = "'{}'".format(val['title'])
    content = "'{}'".format(val['content'].replace('\n', ''))
    date = "'{}'".format(val['written_date'])

    sql = 'INSERT INTO Article(area, title, content, written_date) VALUES({},{},{},{});'.format(
        area, title, content, date)

    curs.execute(sql)
    db.commit()

In [6]:
# Seed for User
users = [(fake.email(), fake.password()) for _ in range(0, 200)]

curs = db.cursor()
for user in users:
    ID = "'{}'".format(user[0])
    PW = "'{}'".format(user[1])

    sql = 'INSERT INTO User(id,pw) VALUES({},{});'.format(ID, PW)
    curs.execute(sql)
    db.commit()

# View seed
## 1. 기사는 5612개
## 2. 유저는 200명
## 3. 유저 한명당 기사를 개별적으로 50 ~ 100개 봤다 가정

In [7]:
import random

curs = db.cursor()
for user_id in range(1,201):
    # 유저가 기사를 총 몇개나 봤는지 (중복 X)
    how_many_articles = random.randrange(50,101)
    
    view_article_list = set()
    
    
    # 그 기사들의 ID
    while len(view_article_list) < how_many_articles:
        view_article_list.add(random.randrange(1,5613))
    
    view_article_list = list(view_article_list)
    view_article_list = [(each,str(fake.date_time_this_year()) ) for each in view_article_list]
    
    for tok in view_article_list:
        article_id = tok[0]
        view_date = "'{}'".format(tok[1])
        
        sql = 'INSERT INTO VIEW(user_id, article_id, view_date) VALUES({},{},{})'.format(user_id, article_id, view_date)
        curs.execute(sql)
        db.commit()